In [ ]:
#사전 구축 함수수
from nltk import FreqDist
import numpy as np
import re

def buildDict(docs):
    doc_tokens = []     # python list
    for doc in docs:
        delim = re.compile(r'[\s,.]+')
        tokens = delim.split(doc.lower()) 
        if tokens[-1] == '' :   tokens = tokens[:-1] 
        doc_tokens.append(tokens)

        
    vocab = FreqDist(np.hstack(doc_tokens))
    vocab = vocab.most_common()
    word_to_id = {word[0] : id for id, word in enumerate(vocab)}
    id_to_word = {id : word[0] for id, word in enumerate(vocab)}
    return doc_tokens, vocab, word_to_id, id_to_word

In [ ]:
#TFID 함수 
from collections import Counter
import math
import numpy as np

def TFIDF(doc_tokens, id_to_word, word_to_id):
    tf_vectors = []
    idf = {}

    #TF 구하기
    for doc in doc_tokens:
        vec = [0.0 for _ in range((len(id_to_word)))]
        word_count = Counter(doc)
        for key, value in word_count.items():
            vec[word_to_id[key]] = 1+ math.log2(value) #tf계산
        tf_vectors.append(vec)
    
    #IDF 구하기
    for id, _ in id_to_word.items():
        idf[id] = 0.0
        for doc in tf_vectors:
            if doc[id] > 0:
                idf[id] += 1
    N = len(tf_vectors)            
    idf = {id : math.log2(N/val) for id, val in idf.items()}

    #TF-IDF 구하기
    idf_list = [val for _, val in idf.items()]
    tfidf = np.array([np.multiply(tf, idf_list) for tf in tf_vectors])

    return tf_vectors, idf, tfidf

In [ ]:
docs = []
docs.append('To do is to be. To be is to do.')
docs.append('To be or not to be. I am what I am')
docs.append('I think therefore I am. Do be do be do.')
docs.append('Do do do da da da. Let it be let it be.')

doc_tokens, vocab, word_to_id, id_to_word = buildDict(docs)
tf_vectors, idf, tfidf = TFIDF(doc_tokens, id_to_word, word_to_id)